In [1]:
# Packages

#pip install -U sentence-transformers
from termcolor import colored
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import numpy as np
import string
import regex as re
import zipfile
import gc
from scipy.stats import boxcox
import sys 
from collections import Counter 
from tqdm import tqdm 
import pandas as pd 
import numpy as np 
import warnings, math
from termcolor import colored
import pickle
import string

# for eval
import random


# for SVD
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix
pd.set_option('display.float_format','{:.5f}'.format)

# for EMB
import scipy
from sklearn import preprocessing 

# for TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy
from sklearn import preprocessing 

# for creating a network
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
EXT_FEATURES_PATH = 'D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Graph_based\ext_features_agg.csv'
NEW_DF_PATH = 'D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Graph_based\Oct_28_df.csv'
PROJECTS_DF_PATH = "D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\projects.csv"


ext_features = pd.read_csv(EXT_FEATURES_PATH)
df = pd.read_csv(NEW_DF_PATH)
projects_df = pd.read_csv(PROJECTS_DF_PATH)


In [3]:
# Donor df----------------------------------------------------------

df_ = pd.merge(df, ext_features[['zip_first_three_digits', 'kmeans_cluster']],
             on='zip_first_three_digits', how='inner')

user_profile_df = df_.groupby('Donor ID'). agg(
    {
    'zip_first_three_digits': 'first',
    'Donor State': lambda x: ", ".join(x),
     'kmeans_cluster': 'first',
     'Donation Amount' : ['sum', 'median'],
     'Project Cost': 'median', 
     'Project Subject Category Tree' : lambda x: ", ".join(x), 
     'Project ID' : lambda x: ",".join(x), 
     'School Metro Type' : lambda x: ",".join(x),
     'Project Grade Level Category' : lambda x:",".join(x),
     'Donation ID' : 'count'

    }
).reset_index()

In [4]:
user_profile_df.head(1)

Donor ID zip_first_three_digits  \
                                                     first   
0  009d5fc7b87883ffad248db5150bf1fc                    782   

                                         Donor State kmeans_cluster  \
                                            <lambda>          first   
0  Texas, Texas, Texas, Texas, Texas, Texas, Texa...              1   

  Donation Amount         Project Cost  \
              sum  median       median   
0       200.95590 0.44020    591.50000   

                       Project Subject Category Tree  \
                                            <lambda>   
0  Literacy & Language, Music & The Arts, Literac...   

                                          Project ID  \
                                            <lambda>   
0  004a152bbe8952ea5e9d5ef89c179933,004a152bbe895...   

                                   School Metro Type  \
                                            <lambda>   
0  urban,urban,urban,urban,urban,urban,suburban,u...   

                        Project Grade Level Category Donation ID  
                                            <lambda>       count  
0  Grades 9-12,Grades 9-12,Grades 9-12,Grades 9-1...         393

In [5]:
def organize_features(text):
    cleaned_features = [i.strip() for i in text.split(",")]
    return dict(Counter(cleaned_features))


In [6]:

user_profile_df['subject_categories'] = user_profile_df['Project Subject Category Tree']['<lambda>'].apply(organize_features)
user_profile_df['grades'] = user_profile_df['Project Grade Level Category']['<lambda>'].apply(organize_features)
user_profile_df['metro_types'] = user_profile_df['School Metro Type']['<lambda>'].apply(organize_features)
user_profile_df['states_donated_to'] = user_profile_df['Donor State']['<lambda>'].apply(organize_features)
user_profile_df['donated_projects'] = user_profile_df['Project ID']['<lambda>']
user_profile_df['donor_zip_first_three_digits'] = user_profile_df['zip_first_three_digits']['first']
user_profile_df['donor_kmeans_cluster'] = user_profile_df['kmeans_cluster']['first']
user_profile_df['donation_amount_sum'] = user_profile_df['Donation Amount']['sum']
user_profile_df['donation_amount_median'] = user_profile_df['Donation Amount']['median']
user_profile_df['project_cost_median'] = user_profile_df['Project Cost']['median']

user_profile_df = user_profile_df.drop(['Project Grade Level Category',
                                  'Project Subject Category Tree', 
                                  'School Metro Type', 'kmeans_cluster',
                                  'Donation ID', 'zip_first_three_digits',
                                  'Project ID', 'Donation Amount',
                                  'Project Cost', 'Donor State'], 
                                 axis=1)

user_profile_df.head(2)

C:\Users\Lenovo\anaconda3\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,Donor ID,subject_categories,grades,metro_types,states_donated_to,donated_projects,donor_zip_first_three_digits,donor_kmeans_cluster,donation_amount_sum,donation_amount_median,project_cost_median
,,,,,,,,,,,
0,009d5fc7b87883ffad248db5150bf1fc,"{'Literacy & Language': 189, 'Music & The Arts...","{'Grades 9-12': 17, 'Grades PreK-2': 195, 'Gra...","{'urban': 288, 'suburban': 105}",{'Texas': 393},"004a152bbe8952ea5e9d5ef89c179933,004a152bbe895...",782,1,200.95590,0.44020,591.50000
1,011ca63acc31287a150a693d92235160,"{'Literacy & Language': 24, 'Music & The Arts'...","{'Grades PreK-2': 39, 'Grades 6-8': 22, 'Grade...","{'urban': 40, 'suburban': 35}",{'North Carolina': 75},"010732a68a6a6a7a40b6827355bd2a04,02180b9e9229c...",295,0,36.03670,0.44020,451.20000


In [7]:
user_profile_df = user_profile_df.droplevel(level=1, axis=1)

In [8]:
user_profile_df.head(1)

,Donor ID,subject_categories,grades,metro_types,states_donated_to,donated_projects,donor_zip_first_three_digits,donor_kmeans_cluster,donation_amount_sum,donation_amount_median,project_cost_median
0,009d5fc7b87883ffad248db5150bf1fc,"{'Literacy & Language': 189, 'Music & The Arts...","{'Grades 9-12': 17, 'Grades PreK-2': 195, 'Gra...","{'urban': 288, 'suburban': 105}",{'Texas': 393},"004a152bbe8952ea5e9d5ef89c179933,004a152bbe895...",782,1,200.95590,0.44020,591.50000


In [10]:
USER_PROFILE_PATH = r'D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Graph_based\user_profile.csv'
user_profile_df.to_csv(USER_PROFILE_PATH, index=False)